# Evolving a PDF

## Method 1: Using eko and ekomark.apply_pdf

In this first part we will compute the eko and subsequently apply the initial PDF "maually" calling a dedicated function. 

In [1]:
import eko

Again, we need to recompute the operator, since we didn't save it.

In [2]:
from banana.data.theories import default_card as th_card
from ekomark.data.operators import default_card as op_card

# here we replace the grid with a very minimal one, to speed up the example
op_card["interpolation_xgrid"] = [1e-3, 1e-2, 1e-1, 5e-1, 1.]
# let's compute two operators: one reaching up to Q2=100 GeV^2 and a second reaching up to Q2=10000 GeV^2
op_card["Q2grid"] = [100., 10000.]
evolution_operator = eko.run_dglap(th_card, op_card)

Next, we need to load the PDF that we want to evolve. EKO uses the same interface as lhapdf to query for the actual values of PDFs. However, for the scope of this tutorial we want to avoid the complication of dealing with an external dependency. Therefore we will use the toy PDFs as they were established by the Les Houches benchmark setting.

In [3]:
from banana import toy
pdf = toy.mkPDF("",0)

Now, we have all ingredients at hand to evolve the PDF set with the operator:

In [4]:
from ekomark.apply import apply_pdf
evolved_pdfs = apply_pdf(evolution_operator, pdf)

The returned object `evolved_pdfs` is a dictionary which maps the requested final scales onto further dictionaries:

In [5]:
evolved_pdfs.keys()

dict_keys([100.0, 10000.0])

Each final scale contains a dictionary with a key `pdfs`, where all PDF values are hold, and a key `errors` , where the integration errors are hold.
Finally, inside each of those there is a mapping of Monte Carlo particle identifiers onto a the PDF value at the requested interpolation points.

E.g. to access the gluon PDF at `Q^2 = 100 GeV^2` you can run:

In [6]:
evolved_pdfs[100.0]["pdfs"][21]

array([ 2.17034797e+04,  3.40247532e+02,  6.64325189e+01, -2.09652559e+01,
        0.00000000e+00])

Note that we return the actual PDF and not the momentum fraction times the PDF.

## Method 2: Using ekobox

In this second part we illustrate how to get (and install) directly a LHAPDF set evolved with eko. 

First, we define our initial pdf. Here, we will use the same toy PDF as in the previous example, but any LHAPDF-like object will do the job!

In [7]:
from banana import toy
pdf = toy.mkPDF("",0)

Now, we set the theory inputs: in this example we will evolve our toy pdf at LO and create a new LHAPDF object with
a size two `Q2grid`.

In [8]:
from banana.data.theories import default_card as th_card

# here we replace the grid with a very minimal one, to speed up the example
op_card["interpolation_xgrid"] = [1e-3, 1e-2, 1e-1, 5e-1, 1.]
op_card["Q2grid"] = [100., 10000.]
# set QCD LO evolution
th_card["PTO"] = 0

Finally, we are ready to run eko and install the new pdf set.
Note that if the evolved pdf already exist the code will raise an error and return.

Additionally, you can set `path` to load a precomputed EKO, while setting `store_path` you can save the produced EKO and reuse it later.
You can also iterate on the given pdf objects.

In [9]:
from ekobox.evol_pdf import evolve_pdfs
evolve_pdfs(
    [pdf],
    th_card,
    op_card,
    install=True,
    name="Evolved_PDF",
)

install_pdf Evolved_PDF


Now, you can access the evolved pdf as all the other PDF sets (note that this requires the Python interface of lhapdf).

In [10]:
import lhapdf
evolved_pdf = lhapdf.mkPDF("Evolved_PDF", 0)

LHAPDF 6.4.0 loading /home/felix/local/share/LHAPDF/Evolved_PDF/Evolved_PDF_0000.dat
Evolved_PDF PDF set, member #0, version 1


To obtain the value of the gluon pdf at a given scale you can simply do:

In [11]:
pid = 21 # gluon id
Q2 = 89.10 #  Q^2 in Gev^2
x = 0.01 # momentum fraction 

# check that the particle is present
print("has gluon?",evolved_pdf.hasFlavor(pid))
# now do the lookup
xg = evolved_pdf.xfxQ2(pid, x, Q2)
print(f"xg(x={x}, Q2={Q2}) = {xg}")

has gluon? True
xg(x=0.01, Q2=89.1) = 3.4255366354811705
